In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_171.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [4]:
sc = SparkContext(master="local[*]", appName="Tweet Count")

# 1s de batchDuration
ssc = StreamingContext(sc, 1)

ssc.checkpoint("checkpoint")

In [5]:
kafkaStream = KafkaUtils.createStream(ssc,zkQuorum='localhost:2181',groupId="group_1" , topics={'kafkaTwitter':2})

In [6]:
# Ventana de 10 segundos.
# Slide de 5.
count_windowed = kafkaStream.countByWindow(10, 5).map(lambda x:('Nº de Tweets tras 5 segundos: %s' % x))

count_windowed.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-01-11 13:24:17
-------------------------------------------

-------------------------------------------
Time: 2020-01-11 13:24:22
-------------------------------------------

-------------------------------------------
Time: 2020-01-11 13:24:27
-------------------------------------------
Nº de Tweets tras 5 segundos: 33

-------------------------------------------
Time: 2020-01-11 13:24:32
-------------------------------------------
Nº de Tweets tras 5 segundos: 129

-------------------------------------------
Time: 2020-01-11 13:24:37
-------------------------------------------
Nº de Tweets tras 5 segundos: 194

-------------------------------------------
Time: 2020-01-11 13:24:42
-------------------------------------------
Nº de Tweets tras 5 segundos: 197

-------------------------------------------
Time: 2020-01-11 13:24:47
-------------------------------------------
Nº de Tweets tras 5 segundos: 190

--------------------------

## ¿Alrededor de qué número (aproximado) se estabiliza el número de tweets que se procesan en el lapso de tiempo indicado (10 segundos)? ¿Tiene sentido? ¿Por qué?

Las primeras ventanas no tienen registros porque el job que mete datos en el topic no esta activo.

la 3º ventana recoge los primeros registro. Como calcula el sumatorio de los últimos 10 segundos tiene 0 registros para los primeros segundos muestra un número inusualmente bajo: 33.

A partir de la 5º ventana se estabilizan los registros en ~ 190 registros cada 10 segundos.


Tiene sentido puesto que según se desplace la ventana llega a cubrir ventanas con registros por cada segenudo. Las primeras tienen 0 registros en cada segundo procesado y según avanza tienen registros para cada uno de los segundos (batches de 1s.)